In [1]:
#libraries
from ortools.linear_solver import pywraplp as glp

In [2]:
#Part 2:
#inputs
demand = [400,200,300] #units
supply = [500,400]#units
unit_cost=[[10,20,15],[12,15,18]]
county=['Hamilton','Butler','Clermont']
supplier=['Southern Gas','Northwest Gas']

In [3]:
#Create model
model2 = glp.Solver('tri-County',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [4]:
#Decision variables:
ship=list(range(len(supplier)))
for s in range(len(supplier)):
    ship[s]=list(range(len(county)))
    for c in range(len(county)):
        ship[s][c]=model2.NumVar(0,model2.infinity(),supplier[s]+'-'+county[c])
    

In [5]:
#Objective function
TotCost=model2.Objective()
TotCost.SetMinimization()
for s in range(len(supplier)):
    for c in range(len(county)):
        TotCost.SetCoefficient(ship[s][c],unit_cost[s][c])


In [6]:
#supply constraints
supply_constr=list(range(len(supplier)))
for s in range(len(supplier)):
    supply_constr[s]=model2.Constraint(0,supply[s],supplier[s])
    for c in range(len(county)):
        supply_constr[s].SetCoefficient(ship[s][c],1)
#demand constraints
demand_constr=list(range(len(county)))
for c in range(len(county)):
    demand_constr[c]=model2.Constraint(demand[c],model2.infinity(),county[c])
    for s in range(len(supplier)):
        demand_constr[c].SetCoefficient(ship[s][c],1)

In [7]:
#Solve model and display result
statusls = ['Optimal','Feasible','Infeasible','Unbounded','Abnormal','Not Solved']
status = model2.Solve()
print('Solution status:', statusls[status])
print('Number of variables:', model2.NumVariables())
print('Number of constraints:',model2.NumConstraints())
print('Total cost: $%.02f'%(TotCost.Value()))

Solution status: Optimal
Number of variables: 6
Number of constraints: 5
Total cost: $11900.00


In [8]:
#Display shipment
for s in range(len(supplier)):
    print(supplier[s])
    for c in range(len(county)):
        print(' \t %s \t %5.1f'%(county[c],ship[s][c].solution_value()))

Southern Gas
 	 Hamilton 	 200.0
 	 Butler 	   0.0
 	 Clermont 	 300.0
Northwest Gas
 	 Hamilton 	 200.0
 	 Butler 	 200.0
 	 Clermont 	   0.0


In [9]:
#Display supplier constraints info
print('Supplier \t LHS \t\t RHS \t\t Slack \t Dual')
LHS = model2.ComputeConstraintActivities()
for s in range(len(supplier)):
    slack = supply[s]-LHS[s]
    print('%s \t %.2f \t %.2f \t %.2f \t %.2f'% (supplier[s],LHS[s],supply[s],slack,supply_constr[s].dual_value()))

Supplier 	 LHS 		 RHS 		 Slack 	 Dual
Southern Gas 	 500.00 	 500.00 	 0.00 	 -2.00
Northwest Gas 	 400.00 	 400.00 	 0.00 	 0.00


In [10]:
#Display demand constraints info
print('County \t\t LHS \t\t RHS \t\t Slack \t Dual')
for c in range(len(county)):
    slack = demand[c]-LHS[c+2]
    print('%7s \t %.2f \t %.2f \t %.2f \t %.2f'% (county[c],LHS[c+2],demand[c],slack,demand_constr[c].dual_value()))

County 		 LHS 		 RHS 		 Slack 	 Dual
Hamilton 	 400.00 	 400.00 	 0.00 	 12.00
 Butler 	 200.00 	 200.00 	 0.00 	 15.00
Clermont 	 300.00 	 300.00 	 0.00 	 17.00


In [11]:
#b.
#Case 1: assume the company contract with Southern Gas to supply the additional demand
demand_constr[1].SetLb(demand[1]+100)
supply_constr[0].SetUb(supply[0]+100)
status = model2.Solve()
print('Solution status:', statusls[status])
print('Total cost: $%.02f'%(TotCost.Value()))
for s in range(len(supplier)):
    print(supplier[s])
    for c in range(len(county)):
        print(' \t %s \t %5.1f'%(county[c],ship[s][c].solution_value()))
#Case 2: assume the company contract with Northwest Gas to supply the additional demand
supply_constr[0].SetUb(supply[0])
supply_constr[1].SetUb(supply[1]+100)
status = model2.Solve()
print('Solution status:', statusls[status])
print('Total cost: $%.02f'%(TotCost.Value()))
for s in range(len(supplier)):
    print(supplier[s])
    for c in range(len(county)):
        print(' \t %s \t %5.1f'%(county[c],ship[s][c].solution_value()))

Solution status: Optimal
Total cost: $13200.00
Southern Gas
 	 Hamilton 	 300.0
 	 Butler 	   0.0
 	 Clermont 	 300.0
Northwest Gas
 	 Hamilton 	 100.0
 	 Butler 	 300.0
 	 Clermont 	   0.0
Solution status: Optimal
Total cost: $13400.00
Southern Gas
 	 Hamilton 	 200.0
 	 Butler 	   0.0
 	 Clermont 	 300.0
Northwest Gas
 	 Hamilton 	 200.0
 	 Butler 	 300.0
 	 Clermont 	   0.0


Since the cost of case 1 < cost of case 2, the company should go with case 1 and contract with Southern Gas